Example based on https://github.com/CybJaz/dyrect

In [7]:
from plyfile import PlyData
import numpy as np

def load_ply_model(filename):
    plydata = PlyData.read(filename)
    coords = np.array([list(x) for x in plydata.elements[0].data])
    return coords

lms = load_ply_model('lorenz_model_20.ply')
lms.shape

(348, 3)

In [8]:
from src import Complex
rips_lms = Complex(lms[:150], 5)
rips_lms.draw_complex()

NameError: name 'np' is not defined